In [1]:
#@title spark startup { form-width: "20%" }
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz

!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

!pip install pyspark




  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=86aff56f362d6ba03b66f12ca64ca88ad80e420a91fd2300643828e470055b77
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"


import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [12]:
import json
import os
from pyspark.sql import functions as F
root = 'drive/MyDrive/MP2-bigdata/data/complete_data'
dataset_path = os.path.join(root, '100_thousand_match_objects')
dataset_path

'drive/MyDrive/MP2-bigdata/data/complete_data/100_thousand_match_objects'

In [13]:
# read the dataset into an rdd and parse json
rdd = sc.textFile( dataset_path).map(lambda s: json.loads(s)).filter(lambda s: 'metadata' in s and 'info' in s).persist()


In [14]:
rdd.take(1)

[{'info': {'gameCreation': 1624666906000,
   'gameDuration': 1489081,
   'gameId': 3956502423,
   'gameMode': 'CLASSIC',
   'gameName': 'teambuilder-match-3956502423',
   'gameStartTimestamp': 1624666953620,
   'gameType': 'MATCHED_GAME',
   'gameVersion': '11.13.382.1241',
   'mapId': 11,
   'participants': [{'assists': 11,
     'baronKills': 0,
     'bountyLevel': 2,
     'champExperience': 12589,
     'champLevel': 14,
     'championId': 516,
     'championName': 'Ornn',
     'championTransform': 0,
     'consumablesPurchased': 5,
     'damageDealtToBuildings': 4367,
     'damageDealtToObjectives': 6489,
     'damageDealtToTurrets': 4367,
     'damageSelfMitigated': 20233,
     'deaths': 1,
     'detectorWardsPlaced': 3,
     'doubleKills': 0,
     'dragonKills': 0,
     'firstBloodAssist': False,
     'firstBloodKill': False,
     'firstTowerAssist': False,
     'firstTowerKill': False,
     'gameEndedInEarlySurrender': False,
     'gameEndedInSurrender': False,
     'goldEarned': 

# separate out match.info from match.meta and save

In [ ]:
#separate the metadata and info parts
meta = rdd.map(lambda l: l['metadata'])
meta_df =meta.toDF()
meta_df.printSchema() # we need to convert participants to string to be able to print
meta_df.show()



/content/spark-2.3.1-bin-hadoop2.7/python/pyspark/sql/session.py:360: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


root
 |-- dataVersion: string (nullable = true)
 |-- matchId: string (nullable = true)
 |-- participants: array (nullable = true)
 |    |-- element: string (containsNull = true)

+-----------+--------------+--------------------+
|dataVersion|       matchId|        participants|
+-----------+--------------+--------------------+
|          2|NA1_3956502423|[L8riBVU9gvIIWlQa...|
|          2|NA1_4046232109|[pR6iBxxmFheXl5Q8...|
|          2|NA1_4075459572|[eyQvk2KUMc_asDWC...|
|          2|NA1_4102861347|[otvuEYXCJbgzn7b7...|
|          2|NA1_4093620767|[fHnVPKUq3XIg3nMG...|
|          2|NA1_4103167521|[dOK_VFFOV3ss9S4T...|
|          2|NA1_4101149930|[TcpSc3wlTvPGK3Tq...|
|          2|NA1_4073079198|[xPl0cuWOSMXXSX4c...|
|          2|NA1_4096678408|[EjexL0Qn9y6IbwYI...|
|          2|NA1_4076126763|[EdiGwmsmdOS1bUaf...|
|          2|NA1_4055293136|[wkevx3_loYV3scui...|
|          2|NA1_3976650608|[VgGxCkdwdRDXq-iX...|
|          2|NA1_4079304372|[8S3aowgTYvE9ibKS...|
|          2|NA1_4092

In [ ]:
match_meta.toPandas().to_csv(dataset_path+'/match_metadata.csv')

In [ ]:
#separate the info rdd 
info = rdd.map(lambda l: l['info'])
info_df = info.toDF().cache()


/content/spark-2.4.4-bin-hadoop2.7/python/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [ ]:
info_df.show()

+-------------+------------+----------+--------+--------------------+------------------+------------+--------------+-----+--------------------+----------+-------+--------------------+--------------+
| gameCreation|gameDuration|    gameId|gameMode|            gameName|gameStartTimestamp|    gameType|   gameVersion|mapId|        participants|platformId|queueId|               teams|tournamentCode|
+-------------+------------+----------+--------+--------------------+------------------+------------+--------------+-----+--------------------+----------+-------+--------------------+--------------+
|1624666906000|     1489081|3956502423| CLASSIC|teambuilder-match...|     1624666953620|MATCHED_GAME|11.13.382.1241|   11|[[bountyLevel -> ...|       NA1|    420|[[objectives ->, ...|              |
|1632017246000|     1513788|4046232109| CLASSIC|teambuilder-match...|     1632017280385|MATCHED_GAME|11.18.395.7538|   11|[[bountyLevel -> ...|       NA1|    420|[[objectives ->, ...|              |
|1634

In [ ]:
N= info_df.count()

In [ ]:
#save_in_chunks(info_df, dataset_path+'/match_info.csv', N)
#info_df.toPandas().to_csv(dataset_path+'/match_info.csv')

# separate team entries from matches.info and save

In [ ]:
# extract out the teams field in the match.info json. I am adding the game id to each team json
def team_mapper(l):
    game= l['gameId']
    teams_list = l['teams']
    [d.update(gameId=game) for d in teams_list ]
    [d.update(gameAndTeam=(game, d['teamId'])) for d in teams_list ]

    return teams_list

teams = info.map(team_mapper)
teams.take(1)[0][0]

{'bans': [{'championId': 360, 'pickTurn': 1},
  {'championId': 164, 'pickTurn': 2},
  {'championId': 55, 'pickTurn': 3},
  {'championId': 122, 'pickTurn': 4},
  {'championId': 24, 'pickTurn': 5}],
 'gameAndTeam': (3956502423, 100),
 'gameId': 3956502423,
 'objectives': {'baron': {'first': True, 'kills': 1},
  'champion': {'first': False, 'kills': 32},
  'dragon': {'first': False, 'kills': 1},
  'inhibitor': {'first': True, 'kills': 1},
  'riftHerald': {'first': True, 'kills': 2},
  'tower': {'first': True, 'kills': 8}},
 'teamId': 100,
 'win': True}

In [ ]:
all_teams = teams.flatMap(lambda l: l)
all_teams_df =all_teams.toDF()
all_teams_df.show()

/content/spark-2.4.4-bin-hadoop2.7/python/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


+--------------------+-----------------+----------+--------------------+------+-----+
|                bans|      gameAndTeam|    gameId|          objectives|teamId|  win|
+--------------------+-----------------+----------+--------------------+------+-----+
|[[pickTurn -> 1, ...|[3956502423, 100]|3956502423|[baron -> [kills ...|   100| true|
|[[pickTurn -> 6, ...|[3956502423, 200]|3956502423|[baron -> [kills ...|   200|false|
|[[pickTurn -> 1, ...|[4046232109, 100]|4046232109|[baron -> [kills ...|   100|false|
|[[pickTurn -> 6, ...|[4046232109, 200]|4046232109|[baron -> [kills ...|   200| true|
|[[pickTurn -> 1, ...|[4075459572, 100]|4075459572|[baron -> [kills ...|   100| true|
|[[pickTurn -> 6, ...|[4075459572, 200]|4075459572|[baron -> [kills ...|   200|false|
|[[pickTurn -> 1, ...|[4102861347, 100]|4102861347|[baron -> [kills ...|   100| true|
|[[pickTurn -> 6, ...|[4102861347, 200]|4102861347|[baron -> [kills ...|   200|false|
|[[pickTurn -> 1, ...|[4093620767, 100]|4093620767|[ba

In [ ]:
all_teams_df.toPandas().to_csv(dataset_path+'/all_teams.csv')

# separate out all patricipants from match.info and save

In [ ]:
# here, I am separating out the participants json field. I add the game ID to the participants json to be 
# able to identify the individual's team and game after I extract the participants column and explode it 
def participants_mapper(l):
    game= l['gameId']
    participants_list = l['participants']
    [d.update(gameId=game) for d in participants_list ]
    [d.update(gameAndTeam=(game, d['teamId'])) for d in participants_list ]

    return participants_list


participants = info.map(participants_mapper)

In [ ]:
#participants_over_all_matches dataframe (parse participants json and explode)
p_all = participants.flatMap(lambda l: l)
p_all_df =p_all.toDF()
p_all_df.show()
#p_all_df.toPandas().to_csv('participants.csv')

/content/spark-2.4.4-bin-hadoop2.7/python/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


+-------+----------+-----------+---------------+----------+----------+------------+-----------------+--------------------+----------------------+-----------------------+--------------------+-------------------+------+-------------------+-----------+-----------+----------------+--------------+----------------+--------------+-----------------+-------------------------+--------------------+----------+----------+---------+------------------+--------------+------------------+--------------+-----+-----+-----+-----+-----+-----+-----+--------------+-------------+-----+------+---------------------+-------------------+----------------+----------------------+----------------+---------------------------+----------------+--------------------+----------+---------+--------------+----------------+-----------------------+-------------+----------+--------------------+-------------------+------------------------------+-------------------+-----------+--------------------+-----------+----------+-----------

In [ ]:
# Reorder the columns to see the ones I'm interested in
def bring_forward(lst, ordered_sublist):
    ordered_sublist.reverse()
    for item in ordered_sublist:
        lst.insert(0, lst.pop(lst.index(item)))

new_order = p_all_df.columns
bring_forward(new_order, ['championId','championName','gameAndTeam' ,'win','summonerName','summonerLevel', 'role', 'lane'])

p_all_df = p_all_df.select(new_order)
p_all_df.show()

+----------+------------+-----------------+-----+---------------+-------------+-------+------+-------+----------+-----------+---------------+----------+-----------------+--------------------+----------------------+-----------------------+--------------------+-------------------+------+-------------------+-----------+-----------+----------------+--------------+----------------+--------------+-------------------------+--------------------+----------+----------+---------+------------------+--------------+------------------+--------------+-----+-----+-----+-----+-----+-----+-----+--------------+-------------+-----+---------------------+-------------------+----------------+----------------------+----------------+---------------------------+----------------+--------------------+----------+---------+--------------+----------------+-----------------------+-------------+----------+--------------------+-------------------+------------------------------+-------------------+-----------+-----------

In [ ]:
p_all_df[['championId']].distinct().count()

157

In [ ]:
# partition the df to avoid collecting the whole thing as we save to CSV 
N = p_all_df.count()

def save_in_chunks(df, filename,N):
    def split_df(df, N1,N2 ):

        df = df.withColumn('INDEX', F.monotonically_increasing_id())
        # sort ascending and take first 100 rows for df1
        df1 = df.sort('INDEX').limit(N1)
        # sort descending and take 400 rows for df2
        df2 = df.sort('INDEX', ascending=False).limit(N2)
        df.drop('INDEX')
        return df1, df2

    chunk1, chunk2 =split_df(df, int(N/2), int(N/2))
    c1,c2 = split_df(chunk1, int(N/4), int(N/4))
    c3,c4 = split_df(chunk2, int(N/4), int(N/4))

    c1.toPandas().to_csv(filename)
    for c in [c2,c3,c4]:
        c.toPandas().to_csv(filename, mode='a', header=False)


In [ ]:
save_in_chunks(p_all_df, 'all_participants.csv',N)

In [ ]:
!cp all_participants.csv drive/MyDrive/MP2-bigdata/data/complete_data

# separate out champion bans data, 




In [ ]:
# extract the bans entry
bans = all_teams.map(lambda l: l['bans'])
bans.sample(True, 0.1).take(2)

[[{'championId': 53, 'pickTurn': 6},
  {'championId': 350, 'pickTurn': 7},
  {'championId': 28, 'pickTurn': 8},
  {'championId': 711, 'pickTurn': 9},
  {'championId': 114, 'pickTurn': 10}],
 [{'championId': 78, 'pickTurn': 1},
  {'championId': 157, 'pickTurn': 2},
  {'championId': 21, 'pickTurn': 3},
  {'championId': 412, 'pickTurn': 4},
  {'championId': 23, 'pickTurn': 5}]]

In [ ]:
# flatten all bans arrays, and group by champion id. then count the ban turns for each champion
from pyspark.sql.types import *
u =F.udf(lambda l: len(l), IntegerType())
champ_bans = bans.flatMap(lambda l: l)
champ_bans_df = champ_bans.toDF().groupby("championId").agg(F.collect_list("pickTurn"))\
                                    .withColumnRenamed("collect_list(pickTurn)","pickTurns")\
                                    .withColumn('totalBans', u('pickTurns'))\
                                    .drop('pickTurns')
champ_bans_df.show()

/content/spark-2.4.4-bin-hadoop2.7/python/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


+----------+---------+
|championId|totalBans|
+----------+---------+
|        26|     2062|
|        29|     2199|
|       222|     1125|
|        19|     3063|
|        54|     4898|
|       112|     1678|
|       113|      132|
|        22|     2828|
|       427|      527|
|         7|    13101|
|        77|      659|
|        34|     3619|
|       126|      998|
|       202|     2869|
|        50|     1946|
|       421|     1867|
|       876|     1441|
|       110|      434|
|        57|      261|
|       136|      194|
+----------+---------+
only showing top 20 rows



In [ ]:
# save the bans dataframe
champ_bans_df.toPandas().to_csv(root+'/champion_bans.csv')


# extract some information from the data dragon

In [ ]:
# translate item number to name by reading from the datadragon
import pandas as pd
import codecs

items_data_dragon = json.load(codecs.open(root+'/data_dragon/items.json', 'r', 'utf-8-sig'))

#extract item number/name dictionary
items_dict = [{'itemIndex':k,'name':v['name'],'tags': v['tags'], 'image': v['image']['full']} for k,v in items_data_dragon.items()]
items_ddragon_pd= pd.DataFrame(items_dict)#.set_index('itemIndex')
items_ddragon_pd.to_csv(root+'/item_names_from_ddragon.csv')
items_ddragon_pd.head()

,itemIndex,name,tags,image
0,1001,Boots,[Boots],1001.png
1,1004,Faerie Charm,[ManaRegen],1004.png
2,1006,Rejuvenation Bead,[HealthRegen],1006.png
3,1011,Giant's Belt,[Health],1011.png
4,1018,Cloak of Agility,[CriticalStrike],1018.png


In [ ]:
# get champion tags/classes by reading from the datadragon
import pandas as pd
import codecs

champions_data_dragon = json.load(codecs.open(root+'/data_dragon/champion.json', 'r', 'utf-8-sig'))

#extract item number/name dictionary
champs_dict = [{'championName':k,'class':v['tags'], 'image': v['image']['full']} for k,v in champions_data_dragon.items()]
champs_ddragon_pd= pd.DataFrame(champs_dict)#.set_index('itemIndex')
champs_ddragon_pd.to_csv(root+'/champion_classes_from_ddragon.csv')
champs_ddragon_pd.head()

,championName,class,image
0,Aatrox,"[Fighter, Tank]",Aatrox.png
1,Ahri,"[Mage, Assassin]",Ahri.png
2,Akali,[Assassin],Akali.png
3,Akshan,"[Marksman, Assassin]",Akshan.png
4,Alistar,"[Tank, Support]",Alistar.png
